In [1]:
from cassandra.cluster import Cluster
import pandas as pd
import numpy as np

In [5]:
cluster = Cluster(["127.0.0.1"],port=9042)


#connect to the keyspace
session = cluster.connect('test')


In [ ]:
#Saving Strategy
#for each new received record
    # 1.check if it's a new client
        # if yes --> save the client and the card and the event 
    # 2.if it's an old client
        # save the event 

In [13]:
# function which checks whether it's a new client or not
def is_new_client(id_client):
    #check if the received id exists in clients table
    user =  pd.DataFrame(list(session.execute(f"SELECT id_client FROM client where \
            id_client = '{id_client}' allow filtering;")))
    if( user.shape[0] == 0):
        return True
    else:
        return False


In [ ]:
#save the new record
def save_new_record(record):

    new_id = record["client"]["id"]

    #if it's a new client
    if(is_new_client(new_id) == True):
        #save the client
        client_id= record["client"]["id"]
        first_name = record["client"]["firstName"]
        last_name = record["client"]["lastName"]
        #last_index='null'
        #top_line='null'
        session.execute(f"INSERT INTO client(id_client, first_name , last_index , last_name ,top_line )\
            VALUES ('{client_id}', '{first_name}',null,'{last_name}',null);")
        #save the card 
        id_card= record["card"]["id"]
        end_of_validity=record["card"]["endOfValidityTimeStamp"]
        type=record["card"]["type"]
        session.execute(f"INSERT INTO card(id_card ,end_of_validity,id_client,type)\
            VALUES ('{id_card}', '{end_of_validity}','{client_id}','{type}');")
        #save the event

            #get the last event_index
        last_index = pd.DataFrame(list(session.execute(f"SELECT max(id_event) as max FROM Event;")))['max']
        index = 0
        if(last_index[0] != None):
            index = int(last_index[0]) + 1


        
        arrival_station = record["destinationStationId"]
        id_card = record["card"]["id"]
        id_user = record["client"]["id"]
        line = record["line"]
        start_station = record["stationId"]
        timestamp = record["requestTimeStamp"]

        session.execute(f"INSERT INTO event(id_event,end_station,id_card,id_user,line,start_station,timestamp)\
            VALUES ({index}, '{arrival_station}','{id_card}','{id_user}','{line}','{start_station}','{timestamp}');")

    else: #if it's an old client
        #save the event
        last_index = pd.DataFrame(list(session.execute(f"SELECT max(id_event) as max FROM Event;")))['max']
        index = 0
        if(last_index[0] != None):
            index = int(last_index[0]) + 1


        
        arrival_station = record["destinationStationId"]
        id_card = record["card"]["id"]
        id_user = record["client"]["id"]
        line = record["line"]
        start_station = record["stationId"]
        timestamp = record["requestTimeStamp"]

        session.execute(f"INSERT INTO event(id_event,end_station,id_card,id_user,line,start_station,timestamp)\
            VALUES ({index}, '{arrival_station}','{id_card}','{id_user}','{line}','{start_station}','{timestamp}');")

In [18]:
is_new_client("f448487")

False

In [ ]:
session.execute(f"INSERT INTO event(id_event,end_station,id_card,id_user,line,start_station,timestamp)\
            VALUES (1, 'Albert 1er de Belgique',2,3,'A','Auguste Delaune','2022-12-25T11:48:26.645064' );")

In [20]:
session.execute(f"INSERT INTO client(id_client, first_name , last_index , last_name ,top_line )\
            VALUES ('f448486', 'Hicham',null,'AHMAD',null);")

In [23]:
last_index = pd.DataFrame(list(session.execute(f"SELECT max(id_event) as max FROM Event;")))['max']